# Novel dataset web scraper

Gathers novel information from novelupdates, i.e. http://www.novelupdates.com/.  
The data is then cleaned and arranged into a dataset.  
The dataset is finally saved as a csv file.

In [66]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from scraper import NovelScraper

# Display all columns when showing dataframes
pd.set_option('display.max_columns', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
version = '0.1.2'

In [68]:
debug = False
scraper = NovelScraper(debug=debug)
novel_info = scraper.parse_all_novels()

Full run, pages with novels: 314
Processing novel: 34622
Processing novel: 2765
Processing novel: 34618
Processing novel: 17893
Processing novel: 11764
Processing novel: 33122
Processing novel: 32889
Processing novel: 18024
Processing novel: 26588
Processing novel: 29443
Processing novel: 34656
Processing novel: 31046
Processing novel: 13538
Processing novel: 34817
Processing novel: 34864
Processing novel: 16868
Processing novel: 23803
Processing novel: 25545
Processing novel: 27453
Processing novel: 34820
Processing novel: 34904
Processing novel: 30540
Processing novel: 7847
Processing novel: 23502
Processing novel: 31859
Processing novel: 4214
Processing novel: 34263
Processing novel: 615
Processing novel: 6363
Processing novel: 35253
Processing novel: 15593
Processing novel: 15752
Processing novel: 31011
Processing novel: 28510
Processing novel: 10432
Processing novel: 34716
Processing novel: 28303
Processing novel: 27810
Processing novel: 31851
Processing novel: 34819
Processing no

Processing novel: 18082
Processing novel: 26967
Processing novel: 33080
Processing novel: 23247
Processing novel: 13148
Processing novel: 16950
Processing novel: 8316
Processing novel: 14190
Processing novel: 33277
Processing novel: 18741
Processing novel: 32728
Processing novel: 20498
Processing novel: 22706
Processing novel: 32747
Processing novel: 33348
Processing novel: 28861
Processing novel: 21312
Processing novel: 34434
Processing novel: 21429
Processing novel: 32901
Processing novel: 12209
Processing novel: 27153
Processing novel: 33564
Processing novel: 24869
Processing novel: 24685
Processing novel: 19020
Processing novel: 24683
Processing novel: 32896
Processing novel: 34276
Processing novel: 2097
Processing novel: 4760
Processing novel: 20367
Processing novel: 21712
Processing novel: 31114
Processing novel: 24678
Processing novel: 288
Processing novel: 10628
Processing novel: 34629
Processing novel: 32421
Processing novel: 16144
Processing novel: 35286
Processing novel: 261

Processing novel: 26073
Processing novel: 12879
Processing novel: 28788
Processing novel: 12795
Processing novel: 35174
Processing novel: 34605
Processing novel: 34990
Processing novel: 30298
Processing novel: 32374
Processing novel: 16504
Processing novel: 32522
Processing novel: 19413
Processing novel: 35082
Processing novel: 33433
Processing novel: 22256
Processing novel: 34261
Processing novel: 34492
Processing novel: 34191
Processing novel: 29753
Processing novel: 16535
Processing novel: 31288
Processing novel: 1124
Processing novel: 34887
Processing novel: 30470
Processing novel: 17981
Processing novel: 32886
Processing novel: 33215
Processing novel: 17258
Processing novel: 33065
Processing novel: 35229
Processing novel: 13111
Processing novel: 32548
Processing novel: 34303
Processing novel: 34149
Processing novel: 3808
Processing novel: 23135
Processing novel: 30403
Processing novel: 28630
Processing novel: 28450
Processing novel: 35296
Processing novel: 24362
Processing novel: 

Processing novel: 29230
Processing novel: 2218
Processing novel: 534
Processing novel: 31286
Processing novel: 20906
Processing novel: 35144
Processing novel: 23221
Processing novel: 31608
Processing novel: 3994
Processing novel: 16693
Processing novel: 19882
Processing novel: 12866
Processing novel: 34725
Processing novel: 3917
Processing novel: 5230
Processing novel: 7701
Processing novel: 31944
Processing novel: 33435
Processing novel: 24590
Processing novel: 15112
Processing novel: 35186
Processing novel: 34581
Processing novel: 35188
Processing novel: 8696
Processing novel: 22609
Processing novel: 33285
Processing novel: 31726
Processing novel: 20467
Processing novel: 33416
Processing novel: 30265
Processing novel: 4606
Processing novel: 21541
Processing novel: 24404
Processing novel: 23272
Processing novel: 10217
Processing novel: 31973
Processing novel: 2679
Processing novel: 33120
Processing novel: 437
Processing novel: 28044
Processing novel: 19013
Processing novel: 24928
Proc

Processing novel: 23520
Processing novel: 34349
Processing novel: 28318
Processing novel: 33745
Processing novel: 15017
Processing novel: 33438
Processing novel: 5504
Processing novel: 4062
Processing novel: 34232
Processing novel: 14
Processing novel: 26548
Processing novel: 22829
Processing novel: 15545
Processing novel: 25016
Processing novel: 34777
Processing novel: 30673
Processing novel: 26799
Processing novel: 31752
Processing novel: 32601
Processing novel: 18384
Processing novel: 20710
Processing novel: 15222
Processing novel: 22294
Processing novel: 34326
Processing novel: 16361
Processing novel: 30322
Processing novel: 27446
Processing novel: 25812
Processing novel: 672
Processing novel: 22267
Processing novel: 22623
Processing novel: 19443
Processing novel: 21188
Processing novel: 16981
Processing novel: 34531
Processing novel: 2052
Processing novel: 32437
Processing novel: 33038
Processing novel: 20113
Processing novel: 35043
Processing novel: 21520
Processing novel: 638
Pr

...

Processing novel: 7319
Processing novel: 12160
Processing novel: 11325
Processing novel: 379
Processing novel: 7750
Processing novel: 12296
Processing novel: 11700
Processing novel: 11716
Processing novel: 9897
Processing novel: 7537
Processing novel: 6070
Processing novel: 8806
Processing novel: 10443
Processing novel: 8482
Processing novel: 9759
Processing novel: 12012
Processing novel: 11961
Processing novel: 11960
Processing novel: 7525
Processing novel: 18667
Processing novel: 4320
Processing novel: 11927
Processing novel: 5633
Processing novel: 10040
Processing novel: 11909
Processing novel: 3983
Processing novel: 11892
Processing novel: 11886
Processing novel: 9180
Processing novel: 8124
Processing novel: 11073
Processing novel: 11811
Processing novel: 11183
Processing novel: 8224
Processing novel: 11581
Processing novel: 7674
Processing novel: 2093
Processing novel: 11809
Processing novel: 11807
Processing novel: 11805
Processing novel: 4380
Processing novel: 6820
Processing no

Processing novel: 8827
Processing novel: 8702
Processing novel: 7030
Processing novel: 7255
Processing novel: 6877
Processing novel: 8804
Processing novel: 8342
Processing novel: 8793
Processing novel: 8791
Processing novel: 8781
Processing novel: 8304
Processing novel: 2340
Processing novel: 8758
Processing novel: 8731
Processing novel: 7904
Processing novel: 1775
Processing novel: 8534
Processing novel: 5817
Processing novel: 164
Processing novel: 6016
Processing novel: 8616
Processing novel: 8531
Processing novel: 8584
Processing novel: 3992
Processing novel: 8590
Processing novel: 8545
Processing novel: 536
Processing novel: 8319
Processing novel: 8250
Processing novel: 1786
Processing novel: 8465
Processing novel: 4124
Processing novel: 7234
Processing novel: 6772
Processing novel: 8389
Processing novel: 4790
Processing novel: 8423
Processing novel: 8335
Processing novel: 8404
Processing novel: 8407
Processing novel: 1291
Processing novel: 4070
Processing novel: 1192
Processing no

Processing novel: 14942
Processing novel: 1515
Processing novel: 4363
Processing novel: 679
Processing novel: 2568
Processing novel: 7373
Processing novel: 742
Processing novel: 1279
Processing novel: 4289
Processing novel: 1153
Processing novel: 4143
Processing novel: 4244
Processing novel: 3257
Processing novel: 2117
Processing novel: 956
Processing novel: 3910
Processing novel: 4101
Processing novel: 1670
Processing novel: 3726
Processing novel: 2211
Processing novel: 2048
Processing novel: 4095
Processing novel: 4083
Processing novel: 4032
Processing novel: 3763
Processing novel: 580
Processing novel: 2967
Processing novel: 3231
Processing novel: 5
Processing novel: 3935
Processing novel: 3948
Processing novel: 140
Processing novel: 3306
Processing novel: 3596
Processing novel: 3517
Processing novel: 338
Processing novel: 3848
Processing novel: 3045
Processing novel: 2277
Processing novel: 3105
Processing novel: 2258
Processing novel: 2883
Processing novel: 157
Processing novel: 27

Processing novel: 3566
Processing novel: 8106
Processing novel: 1187
Processing novel: 2019
Processing novel: 1415
Processing novel: 1313
Processing novel: 1398
Processing novel: 2173
Processing novel: 1907
Processing novel: 1377
Processing novel: 1708
Processing novel: 1264
Processing novel: 3689
Processing novel: 3691
Processing novel: 5585
Processing novel: 18863
Processing novel: 1739
Processing novel: 3805
Processing novel: 4536
Processing novel: 1400
Processing novel: 11405
Processing novel: 17556
Processing novel: 3519
Processing novel: 3798
Processing novel: 3564
Processing novel: 1173
Processing novel: 1175
Processing novel: 1177
Processing novel: 2042
Processing novel: 7145
Processing novel: 21608
Processing novel: 1183
Processing novel: 25482
Processing novel: 18748
Processing novel: 18755
Processing novel: 17229
Processing novel: 1190
Processing novel: 7077
Processing novel: 9241
Processing novel: 10642
Processing novel: 4859
Processing novel: 1198
Processing novel: 4869
Pr

Processing novel: 18764
Processing novel: 29477
Processing novel: 20809
Processing novel: 25552
Processing novel: 22946
Processing novel: 11995
Processing novel: 1985
Processing novel: 1993
Processing novel: 1995
Processing novel: 9157
Processing novel: 12806
Processing novel: 24128
Processing novel: 34439
Processing novel: 2007
Processing novel: 2008
Processing novel: 24687
Processing novel: 19053
Processing novel: 2015
Processing novel: 2017
Processing novel: 7272
Processing novel: 25032
Processing novel: 24971
Processing novel: 2023
Processing novel: 2025
Processing novel: 2035
Processing novel: 4369
Processing novel: 2040


In [69]:
df = pd.DataFrame(novel_info)

if not debug:
    file_name = 'novels_' + version + '.csv'
else:
    file_name = 'novels_debug.csv'
    
df.to_csv(file_name, header=True, index=False)